<a href="https://colab.research.google.com/github/afchatfield/Movie-Database/blob/main/Movie_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cloudpickle as cPickle
with open('/content/gdrive/MyDrive/movie_database.pkl', 'rb') as f:
  movie_database = cPickle.load(f)

**Prepare Web Driver**

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
# wd.get("https://www.webite-url.com")

     |████████████████████████████████| 954 kB 4.1 MB/s 
     |████████████████████████████████| 138 kB 51.8 MB/s 
     |████████████████████████████████| 356 kB 55.6 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 3.5 MB 34.2 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [ ]:
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


**Get List of Movies From List Challenge**

In [ ]:
from bs4 import BeautifulSoup
wd.get("https://www.listchallenges.com/reddit-top-250-movies-2017-edition/stats")
content = wd.page_source
soup = BeautifulSoup(content, 'html.parser')


In [ ]:
def checkString(input):
  try:
    if int(input):
      return False
  except ValueError:
    if input[-3:]=='...':
      return True

def checkMovieFormat(input):
  title = input.split(' ')
  # check last word is valid format (year in brackets)
  try:
    if (title[-1][0] == '(' and title[-1][5] == ')'):
      try:
        if int(title[-1][1:5]):
          return True
      except ValueError:
        if checkString(input):
          return True
        return False
    if checkString(input):
      return True
    return False
  except IndexError:
    if checkString(input):
      return True
    return False


In [ ]:
# wd.get("https://www.listchallenges.com/reddits-top-250-movies-2019-edition/stats")
# wd.get("https://www.listchallenges.com/reel-stats-statistical-top-250-movies-of-all-time/stats")
# wd.get("https://www.listchallenges.com/best-films-of-the-last-10-years-2011-2021/stats")
wd.get("https://www.listchallenges.com/100-most-popular-foreign-language-films-ever-made/stats")
content = wd.page_source
soup = BeautifulSoup(content, 'html.parser')

In [ ]:
movies = []
for a in soup.findAll('div', "flot-tick-label tickLabel"):
  title = a.text
  if checkMovieFormat(title):
    movies.append(title)

In [ ]:
new_movies = []

existing_movies = []

for movie in movie_database:
  existing_movies.append(movie['original_title'])

for movie in movies:
  if movie in existing_movies:
    continue
  else:
    new_movies.append(movie)


**Get imdb searches**

In [ ]:
google_query = "https://www.google.com/search?q={}"
imdb_query = "https://www.imdb.com/find?q={}&ref_=nv_sr_sm"
bing_query = "https://www.bing.com/search?q={}"
justwatch_query = "https://www.justwatch.com/uk/search?q={}"

In [ ]:
import requests
from tqdm import tqdm
from urllib3.exceptions import MaxRetryError

def get_movie_query(website, movie):
  movie_words = movie.split(' ')
  movie_query = ''
  for word in movie_words:
    movie_query += word.lower()
    movie_query += '+'
  movie_query = movie_query[:-1]
  # return "https://www.imdb.com/find?q={}&ref_=nv_sr_sm".format(movie_query)
  return website.format(movie_query)

def get_imdb_page(movie):
  # get query page from movie name
  query = get_movie_query(imdb_query, movie)
  
  # get page content 
  wd.get(query)
  content = wd.page_source
  soup = BeautifulSoup(content, 'html.parser')
  for result in soup.findAll('td', "result_text"):
    title = result.text.lstrip().rstrip()
    if title == movie:
      return 'https://www.imdb.com{}'.format(result.find('a')['href'])

  return None

# get_imdb_page('Ran (1985)')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


**Collate data from bing and ratings websites**

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def getInfoFromUtilAns(info, movie):
  # get title
  title = info.find('span').text
  # title = info.find('span', attrs={'class':'l_ecrd_txt_heroxs'}).text #attrs={'class':'l_ecrd_thumbheader_ttl l_ecrd_txt_small'})
  # get ratings and sites
  ratings = {}
  websites = []
  for rating_cards in info.findAll('div', attrs={'class':'l_ecrd_ratings'}):
    # get rating sites
    website = rating_cards.findAll('a', attrs={'class':'l_ecrd_txt_lnk'})
    for link in website:
      websites.append(link['href'])
    #get ratings
    for site in rating_cards.findAll('div', attrs={'class':'l_ecrd_ratings_txt'}):
      rating = site.find('div', attrs={'class':'l_ecrd_ratings_prim'}).text
      company = site.find('div', attrs={'class':'l_ecrd_txt_qfttl'}).text
      ratings[company] = rating
  # get info from sites
  if websites==[]: #if no websites, get imdb page
    websites.append(get_imdb_page(movie))
  findrating=False
  if ratings=={}: #if no ratings set findrating to true for rating site info
    findrating=True
  #get imdb page if rotten and set imdrating to true
  if getDomainName(websites[0])!='imdb':
    try:
      imdb_page = get_imdb_page(movie)
      if imdb_page:
        websites.insert(0, imdb_page)
        findrating=True
    except:
      print("Couldn't find IMDb page, using rottentomatoes")
  #get movie info from imdb
  movie_info = getInfoFromRatingSites(websites[0], findrating=findrating)
  if findrating:
    ratings.update(movie_info['ratings'])
  return {'title':title, 'year':movie_info['year'], 'ratings':ratings, 'time':movie_info['time'], 
          'genres':movie_info['genres'], 'plot':movie_info['plot'], 'age_rating':movie_info['age_rating'],
          'director':movie_info['director'], 'sites':websites, 'original_title':movie}

def getInfoByWaitingForElement(movie):
  query = get_movie_query(bing_query, movie)
  wd.get(query)
  try:
      utilAnsElem = WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/aside/ol/li[1]/div")))
  except TimeoutException:
      return "Loading took too much time!"
  utilAnsHTML = utilAnsElem.get_attribute('innerHTML')
  soup = BeautifulSoup(utilAnsHTML, 'html.parser')
  return getInfoFromUtilAns(soup, movie)

getInfoByWaitingForElement(new_movies[0])

{'age_rating': 'Not Rated',
 'director': 'Anurag Kashyap',
 'genres': ['Action', 'Comedy', 'Crime'],
 'original_title': 'Gangs of Wasseypur (2012)',
 'plot': 'A clash between Sultan and Shahid Khan leads to the expulsion of Khan from Wasseypur, and ignites a deadly blood feud spanning three generations.A clash between Sultan and Shahid Khan leads to the expulsion of Khan from Wasseypur, and ignites a deadly blood feud spanning three generations.A clash between Sultan and Shahid Khan leads to the expulsion of Khan from Wasseypur, and ignites a deadly blood feud spanning three generations.',
 'ratings': {'IMDb': '8.2/10', 'Metacritic': '89', 'Rotten Tomatoes': '97%'},
 'sites': ['http://www.imdb.com/title/tt1954470/',
  'https://www.rottentomatoes.com/m/gangs_of_wasseypur?search=gangs%20of/',
  'https://www.metacritic.com/movie/gangs-of-wasseypur/details'],
 'time': '5h 21min',
 'title': 'Gangs of Wasseypur',
 'year': 2012}

In [ ]:
# movie = "La Dolce Vita (1960)"

def getMovieInfo(movie):
  query = get_movie_query(bing_query, movie)
  wd.get(query)
  soup = BeautifulSoup(wd.page_source, 'html.parser')
  for result in soup.findAll('ol', attrs={'id':"b_context"}):
      if result.find('div', attrs={'class':'b_entityBP'}):
        # get movie info 
        info = result.find('div', attrs={'class':'b_entityBP'})
        # get title
        title = info.find('h2', attrs={'class':' b_entityTitle'}).result_text
      elif result.find('div', attrs={'class':'utilAns'}):
        # get movie info
        info = result.find('div', attrs={'class':'utilAns'})
        return getInfoFromUtilAns(info)
      else:
        return getInfoByWaitingForElement(movie)
      # return {'title':title, 'year':movie_info['year'], 'ratings':ratings, 'time':movie_info['time'], 
      #         'genres':movie_info['genres'], 'plot':movie_info['plot'], 'age_rating':movie_info['age_rating'],
      #         'director':movie_info['director'], 'sites':websites, 'original_title':movie}
getMovieInfo('les intouchables')

**Get Data from Rating sites (imdb only atm)**

In [ ]:
def getDomainName(website):
  domain = website.split('//')[1].split('/')[0]
  return domain.split('.')[1]

def getInfoFromRatingSites(site, findrating=False):
  domain_name = getDomainName(site)
  if domain_name=='imdb':
    return getDataFromIMDB(site, findrating=findrating)
  elif domain_name == 'metacritic':
    return "Metacritic"
  elif domain_name == 'rottentomatoes':
    return "rottentomatoes"

def getDataFromIMDB(site, findrating=False):
  wd.get(site)
  soup = BeautifulSoup(wd.page_source, 'html.parser')

  # get year, age rating and length of film
  try:
    year_age_time = soup.find('ul', attrs={'class':'ipc-inline-list ipc-inline-list--show-dividers TitleBlockMetaData__MetaDataList-sc-12ein40-0 dxizHm baseAlt'}).findAll('li')
    year = year_age_time[0].find('span').text
    age_rating = year_age_time[1].find('a')
    if age_rating==None:
      age_rating = year_age_time[1].find('span')
    age_rating = age_rating.text
    time = year_age_time[2].text
  except AttributeError:
    year_age_time = soup.find('div', attrs={'class':'TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2 hWHMKr'}).findAll('li', attrs={'class':'ipc-inline-list__item'})
    year = year_age_time[0].find('span').text
    time = year_age_time[1].text
  movie_info = {'year': int(year), 'age_rating':age_rating, 'time':time}

  # get genres and plot
  genre_plot = soup.find('div', attrs={'class':'GenresAndPlot__ContentParent-cum89p-8 bFvaWW Hero__GenresAndPlotContainer-kvkd64-11 twqaW'})
  if genre_plot==None:
      genre_plot = soup.find('div', attrs={'class':'GenresAndPlot__OffsetContentParent-cum89p-9 dUAPpa Hero__GenresAndPlotContainer-kvkd64-11 twqaW'})
  genre_buttons = genre_plot.findAll('a', attrs={'class':'GenresAndPlot__GenreChip-cum89p-3 fzmeux ipc-chip ipc-chip--on-baseAlt'})
  genres = [genre.text for genre in genre_buttons]
  movie_info['genres'] = genres
  plot = genre_plot.find('p', attrs={'data-testid':'plot'}).text
  movie_info['plot'] = plot

  # get director
  director_data = soup.findAll('li', attrs={'class':'ipc-metadata-list__item'})[0]
  movie_info['director'] = director_data.text[8:]

  # get rating if findrating=true
  rating = soup.find('span', attrs={'class':'AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV'})
  movie_info['ratings'] = {'IMDb':rating.text}

  return movie_info

# getInfoFromRatingSites('https://www.imdb.com/title/tt0091251/?ref_=fn_al_tt_1', findrating=True)
# getInfoFromRatingSites('https://www.metacritic.com/movie/star-wars-episode-iv---a-new-hope/details')

**Add movie data to database**

In [ ]:
from tqdm import tqdm
def addToMovieDatabase(database, movies):
  movie_database = database
  for movie in tqdm(movies):
    try:
      info = getMovieInfo(movie)
      if info != 'nothing':
        movie_database.append(info)
      else:
        print("nothing", movie)
    except:
      print(movie)
  return movie_database

In [ ]:
getMovieInfo(new_movies[0])

{'age_rating': 'PG-13',
 'director': 'Hirokazu Koreeda',
 'genres': ['Drama'],
 'original_title': 'Nobody Knows (2004)',
 'plot': 'In a small Tokyo apartment, twelve-year-old Akira must care for his younger siblings after their mother leaves them and shows no sign of returning.In a small Tokyo apartment, twelve-year-old Akira must care for his younger siblings after their mother leaves them and shows no sign of returning.In a small Tokyo apartment, twelve-year-old Akira must care for his younger siblings after their mother leaves them and shows no sign of returning.',
 'ratings': {'IMDb': '8.1/10', 'Metacritic': '88', 'Rotten Tomatoes': '93%'},
 'sites': ['http://www.imdb.com/title/tt0408664/',
  'https://www.rottentomatoes.com/m/nobody_knows/',
  'https://www.metacritic.com/movie/nobody-knows/details'],
 'time': '2h 21min',
 'title': 'Nobody Knows',
 'year': 2004}

In [ ]:
movie_database = addToMovieDatabase(movie_database, new_movies)

 62%|██████▏   | 74/119 [06:14<03:08,  4.19s/it]

Fanny & Alexander (1982)


100%|██████████| 119/119 [09:59<00:00,  5.04s/it]


**Get Streaming data from justwatch**

In [ ]:
import time
def findJustwatchLink(movie, og_movie, year):
  time.sleep(0.005)
  og_movie = og_movie[:-7]
  query = get_movie_query(justwatch_query, movie)
  wd.get(query)
  soup = BeautifulSoup(wd.page_source, 'html.parser')
  found = False
  for results in soup.findAll('ion-row', attrs={'class':'title-list-row__row md hydrated'}):
    if found:
      break
    for film in results.findAll('ion-col', attrs={'class':'md hydrated'}):
      if found:
        break
      title = film.find('span', attrs={'class':'title-list-row__row__title'})
      year = film.find('span', attrs={'class':'title-list-row__row--muted'})
      if title==None:
        continue
      title = title.text
      if title==movie:
        link = film.find('a')['href'] 
        return 'https://www.justwatch.com'+link
      elif title==og_movie:
        link = film.find('a')['href'] 
        return 'https://www.justwatch.com'+link
      elif (og_movie in title or movie in title) and year==year.text:
        link = film.find('a')['href'] 
        return 'https://www.justwatch.com'+link  
  return None

def findMovieStreams(movie_info):
  streams = []
  query = findJustwatchLink(movie_info['title'], movie_info['original_title'], movie_info['year'])
  if query == None:
    query = findJustwatchLink(movie_info['title'], movie_info['original_title'], movie_info['year'])
  if query == None:
    return None
  wd.get(query)
  soup = BeautifulSoup(wd.page_source, 'html.parser')

  for services in soup.findAll('div', attrs={'class': 'price-comparison__grid__row price-comparison__grid__row--stream'}):
    for service in services.findAll('div', attrs={'class':'price-comparison__grid__row__element'}):
      for img in service.findAll('img', attrs={'class':' price-comparison__grid__row__icon'}):
        streams.append(img['title'])
  return streams

# findMovieStreams(movie_database[10])

In [ ]:
from tqdm import tqdm
for movie in tqdm(movie_database):
  if 'streams' in movie:
    pass
  else:
    movie['streams'] = findMovieStreams(movie)


100%|██████████| 429/429 [10:22<00:00,  1.45s/it]


In [ ]:
movie_database[-11:]

[{'age_rating': 'R',
  'director': 'Ben Stiller',
  'genres': ['Action', 'Comedy', 'War'],
  'imdb_rating': 7.0,
  'original_title': 'Tropic Thunder (2008)',
  'plot': 'Through a series of freak occurrences, a group of actors shooting a big-budget war movie are forced to become the soldiers they are portraying.Through a series of freak occurrences, a group of actors shooting a big-budget war movie are forced to become the soldiers they are portraying.Through a series of freak occurrences, a group of actors shooting a big-budget war movie are forced to become the soldiers they are portraying.',
  'ratings': {'IMDb': '7/10', 'Metacritic': '71', 'Rotten Tomatoes': '82%'},
  'seen': '/',
  'sites': ['http://www.imdb.com/title/tt0942385/',
   'https://www.rottentomatoes.com/m/tropic_thunder?cmpid=rtnewsletter_08_15_08/',
   'https://www.metacritic.com/movie/tropic-thunder/details'],
  'streams': ['Virgin TV Go'],
  'time': '1h 47min',
  'title': 'Tropic Thunder',
  'year': 2008},
 {'age_rat

**Sort films**

In [ ]:
import math

# movie_information = movie_database[57]
# print(movie_information['year'], movie_information['title'])
def find_average_rating(ratings):
  str_values = []
  int_values = []
  for rating in ratings:
    str_values.append(ratings[rating])
  for value in str_values:
    try:
      int_values.append((int(value)))
    except ValueError:
      if value[-1]=='%':
        int_values.append(int(value[:-1]))
      elif value[-3:]=='/10':
        int_values.append(int(float(value[:-3])*10))
      else:
        int_values.append(int(float(value)*10))
  if len(int_values)==0:
    return 0
  return round(sum(int_values)/len(int_values), 1)

def isstreamable(services, available_streams):
  if available_streams==None:
    return False
  return (len([i for i in services if i in available_streams])!=0)

def rightGenres(rightGenres, movieGenres):
  if movieGenres==None:
    return 0
  return (len([i for i in rightGenres if i in movieGenres]))

def getDecade(year):
  return math.floor(year/10)*10

def yearScore(year):
  currentYear = 2021
  currentDecade = getDecade(currentYear)
  filmDecade = getDecade(year)
  return int((currentDecade-filmDecade)/10)

def rate_by_rating_year(year1, year2, rating1, rating2):
  #returns true only if film2 is within 5 rating and is more recent
  rating_difference = abs(rating1-rating2)
  if rating_difference<=5:
    film1decade = yearScore(year1)
    film2decade = yearScore(year2)
    if film1decade<film2decade: #film1 is more recent
      return False
    elif film2decade<film1decade:
      return True
    else:
      return False
  else:
    return False

def compareFilms(film1, film2):
  film1streamable = isstreamable(streams, film1['streams'])
  # film1rating = find_average_rating(film1['ratings'])
  film2streamable = isstreamable(streams, film2['streams'])
  # film2rating = find_average_rating(film2['ratings'])
  if film1streamable:
    if film2streamable:
      return compareSeenFilms(film1, film2)
    else:
      return 1 #film1['title']
  else:
    if film2streamable:
      return -1 #film2['title']
    else:
      return compareSeenFilms(film1, film2)

def compareSeenFilms(film1, film2):
  if film1['seen']=='/':
    if film2['seen']=='/':
      return compareGenres(film1, film2)
    else:
      return -1 #film2
  else:
    if film2['seen']=='/':
      return 1 #film1
    else:
      return compareGenres(film1, film2) 

def compareGenres(film1, film2):
  film1GenreCount = rightGenres(genres, film1['genres'])
  film2GenreCount = rightGenres(genres, film2['genres'])
  if film1GenreCount > film2GenreCount:
    return 1
  elif film2GenreCount > film1GenreCount:
    return -1
  else:
    return compareRatingYearFilms(film1, film2)

def compareRatingYearFilms(film1, film2):
  film1rating = find_average_rating(film1['ratings'])
  film2rating = find_average_rating(film2['ratings'])
  if film1rating>film2rating:
    if rate_by_rating_year(film1['year'], film2['year'], film1rating, film2rating):
      return -1 #film2['title']
    else:
      return 1# film1['title']
  elif film2rating>film1rating:
    if rate_by_rating_year(film2['year'], film1['year'], film2rating, film1rating):
      return 1 #film1['title']
    else:
      return -1 #film2['title']
  else:
    return 0# "Draw both streamable and same rating"

streams = ['Mubi', 'Netflix', 'Disney Plus']#, 'Amazon Prime Video']
genres = []


In [ ]:
from random import randint

# movie_database.sort(key=compareFilms)
def sortMovieDatabase(database):
  # quicksort
  # If the input array contains fewer than two elements,
  # then return it as the result of the function
  if len(database) < 2:
      return database

  low, same, high = [], [], []

  # Select your `pivot` element randomly
  pivot = database[randint(0, len(database) - 1)]

  for movie in database:
      # Elements that are smaller than the `pivot` go to
      # the `low` list. Elements that are larger than
      # `pivot` go to the `high` list. Elements that are
      # equal to `pivot` go to the `same` list.
      if compareFilms(movie, pivot) < 0:
          low.append(movie)
      elif compareFilms(movie, pivot) == 0:
          same.append(movie)
      elif compareFilms(movie, pivot) > 0:
          high.append(movie)

  # The final result combines the sorted `low` list
  # with the `same` list and the sorted `high` list
  return sortMovieDatabase(low) + same + sortMovieDatabase(high)

In [ ]:
for movie in movie_database:
  if 'seen' in movie:
    continue
  else:
    valid_input = False
    while not valid_input:
      seen = input("Have you seen: {}? (/) for yes, '' for no".format(movie['title']))
      if seen=='/' or seen=='':
        movie['seen'] = seen
        valid_input=True
      else:
        print("Please enter a valid input: (/) for yes or (enter) for no")

In [ ]:
def seenMovieByTitle(title):
  for m in movie_database:
    if m['title']==title or m['original_title']==title:
      m['seen'] = '/'
      return True
  return False
seenMovieByTitle('Captain America: Civil War')

True

In [ ]:
movie_dataframe = movie_database.copy()
movie_dataframe = sortMovieDatabase(movie_dataframe)

In [ ]:
movie_dataframe = movie_dataframe[::-1]
movie_database = movie_dataframe
movie_database

[{'age_rating': 'PG',
  'director': 'Isao Takahata',
  'genres': ['Animation', 'Adventure', 'Drama'],
  'original_title': 'The Tale of the Princess Kaguya (2013)',
  'plot': 'Found inside a shining stalk of bamboo by an old bamboo cutter and his wife, a tiny girl grows rapidly into an exquisite young lady. The mysterious young princess enthralls all who encounter... Read allFound inside a shining stalk of bamboo by an old bamboo cutter and his wife, a tiny girl grows rapidly into an exquisite young lady. The mysterious young princess enthralls all who encounter her, but ultimately she must confront her fate, the punishment for her crime.Found inside a shining stalk of bamboo by an old bamboo cutter and his wife, a tiny girl grows rapidly into an exquisite young lady. The mysterious young princess enthralls all who encounter her, but ultimately she must confront her fate, the punishment for her crime.',
  'ratings': {'IMDb': '8/10', 'Metacritic': '89', 'Rotten Tomatoes': '100%'},
  'see

**Restructure dataset to not include nested dictionaries**

In [ ]:
import pandas as pd
movie_ratings = movie_database[1]['ratings']

def dictionaryToString(dictionary):
  string = ''
  for key in dictionary:
    string = string + key+': '+ dictionary[key]+', '
  return string[:-2]

# movie_dataframe = movie_database.copy()
# for i, movie_information in enumerate(movie_dataframe):
#   ratings = movie_information['ratings']
#   if isinstance(ratings, dict):
#     ratings = dictionaryToString(ratings)
#   rating = ratings[6:9]
#   if '/' in rating:
#     rating = rating[0]
#   movie_dataframe[i]['imdb_rating'] = float(rating)

**Create and show dataframe**

In [ ]:
df = pd.DataFrame(movie_dataframe)
df.head()

,title,year,ratings,time,genres,plot,age_rating,director,sites,original_title,streams,imdb_rating,seen
0,8½,1963,"{'IMDb': '8/10', 'Rotten Tomatoes': '98%', 'Me...",2h 18min,[Drama],A harried movie director retreats into his mem...,Not Rated,Federico Fellini,"[http://www.imdb.com/title/tt0056801/, https:/...",8½ (1963),"[Amazon Prime Video, Amazon Prime Video, Mubi,...",8.0,
1,The Tale of The Princess Kaguya,2013,"{'IMDb': '8/10', 'Rotten Tomatoes': '100%', 'M...",2h 17min,"[Animation, Adventure, Drama]",Found inside a shining stalk of bamboo by an o...,PG,Isao Takahata,"[http://www.imdb.com/title/tt2576852/, https:/...",The Tale of the Princess Kaguya (2013),"[Netflix, Netflix Kids]",NaN,
2,Marriage Story,2019,"{'IMDb': '7.9/10', 'Rotten Tomatoes': '94%', '...",2h 17min,"[Comedy, Drama, Romance]",Noah Baumbach's incisive and compassionate loo...,R,Noah Baumbach,"[http://www.imdb.com/title/tt7653254/, https:/...",Marriage Story (2019),[Netflix],NaN,
3,Schindler's List,1993,"{'IMDb': '8.9/10', 'Rotten Tomatoes': '98%', '...",3h 15min,"[Biography, Drama, History]","In German-occupied Poland during World War II,...",R,Steven Spielberg,"[http://www.imdb.com/title/tt0108052/, https:/...",Schindler's List (1993),"[Netflix, Virgin TV Go]",8.9,
4,The Iron Giant,1999,"{'IMDb': '8/10', 'Rotten Tomatoes': '96%', 'Me...",1h 26min,"[Animation, Action, Adventure]",A young boy befriends a giant robot from outer...,PG,Brad Bird,"[http://www.imdb.com/title/tt0129167/, https:/...",The Iron Giant (1999),"[Netflix, Virgin TV Go]",8.0,


# Save as csv file

In [ ]:
df.to_csv('/content/movie_info.csv')

In [ ]:
import cloudpickle as cPickle
with open('/content/gdrive/MyDrive/movie_database.pkl', 'wb') as f:
  cPickle.dump(movie_database, f)

# Flask App

In [ ]:
!pip install flask-ngrok